In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans, KMeans

In [2]:
pd.set_option('display.max_colwidth', None)

Funções auxiliares para este notebook
- load_dataframe_experiment: leitura dos resultados dos experimentos retornando um dataframe
- dataset_sep: Separa o dataset passado como parâmetro em dois, acertos e erros
- print_pecentage_ok_nok: imprime a porcentagem de acertos e erros
- questions_bag_words: recebe o dataframe do experimento, extrai as perguntas e retorna um dataframe bag of words de cada pergunta

In [3]:
def load_dataframe_experiment(expNumber):
    path = '../dataset/experiments/experiment_' + str(expNumber) + '/results/validation_pm.txt'
    df = pd.read_csv(path, sep='\t', header = 0)
    df = df.iloc[:, :-1]
    return df;

def dataset_sep(df_original, filter_value = None):
    if (filter_value != None):
        df_original = df_original.query('hardness == "' + str(filter_value) + '"')
    df_ok = df_original[df_original["exact_match"] == 'OK']
    df_nok = df_original[df_original["exact_match"] == 'NOK']
    return df_original, df_ok, df_nok

def print_percentage_ok_nok(df_total, df_ok, df_nok, description):
    print (description)
    print('OK: ' + str(round(df_ok.shape[0] / df_total.shape[0] * 100, 2)) + '%')
    print('NOK: ' + str(round(df_nok.shape[0] / df_total.shape[0] * 100, 2)) + '%')
    print('--')

def questions_bag_words(df_experiment):
    qt_np = df_experiment["question"].values
    bow = CountVectorizer().fit(qt_np)
    qt_vec = bow.transform(qt_np)
    df_bag_words = pd.DataFrame(qt_vec.toarray(), columns = bow.get_feature_names_out())
    return bow, qt_vec, df_bag_words

def question_id_diff(df1, df2):
    print(np.setdiff1d(df1.loc[:,"question_id"],df2.loc[:, "question_id"]))

Leitura de datasets com enunciados, declaração SQL predita, declaração SQL gold e informações de cada componente da declaração SQL (SELECT, WHERE, ORDER, ..) e o nível de dificuldade classificado pelo Spider (easy, medium, hard, extra)

Datasets
 - Process_mininig (sp_by_db.txt)
 - Spider (spider.txt)

In [10]:
df_experiment_1 = load_dataframe_experiment(1)
df_experiment_2 = load_dataframe_experiment(2)
df_experiment_3 = load_dataframe_experiment(3)
df_experiment_5 = load_dataframe_experiment(5)
df_experiment_5 = df_experiment_5.drop([84, 165])
df_experiment_6 = load_dataframe_experiment(6)
df_experiment_7 = load_dataframe_experiment(7)
df_experiment_8 = load_dataframe_experiment(8)

Separação dos conjuntos a serem analisados em dois conjuntos (OK e NOK)

In [15]:
df_experiment_1, df_experiment_1_ok, df_experiment_1_nok = dataset_sep(df_experiment_1, 'easy')
df_experiment_2, df_experiment_2_ok, df_experiment_2_nok = dataset_sep(df_experiment_2, 'easy')
df_experiment_3, df_experiment_3_ok, df_experiment_3_nok = dataset_sep(df_experiment_3, 'easy')
df_experiment_5, df_experiment_5_ok, df_experiment_5_nok = dataset_sep(df_experiment_5, 'easy')
df_experiment_6, df_experiment_6_ok, df_experiment_6_nok = dataset_sep(df_experiment_6, 'easy')
df_experiment_7, df_experiment_7_ok, df_experiment_7_nok = dataset_sep(df_experiment_7, 'easy')
df_experiment_8, df_experiment_8_ok, df_experiment_8_nok = dataset_sep(df_experiment_8, 'easy')

Imprimindo a porcentagem de cada experimento

In [16]:
print_percentage_ok_nok(df_experiment_1, df_experiment_1_ok, df_experiment_1_nok, 'Todas as tabelas em única base de dados')
print_percentage_ok_nok(df_experiment_2, df_experiment_2_ok, df_experiment_2_nok, 'Única tabela em uma única base de dados')
print_percentage_ok_nok(df_experiment_3, df_experiment_3_ok, df_experiment_3_nok, 'Cada tabela em uma base de dados diferente')
print_percentage_ok_nok(df_experiment_5, df_experiment_5_ok, df_experiment_5_nok, 'Cada tabela em uma base de dados diferente com ajustes')
print_percentage_ok_nok(df_experiment_6, df_experiment_6_ok, df_experiment_6_nok, 'Cada tabela em uma base de dados diferente com paráfrases')
print_percentage_ok_nok(df_experiment_7, df_experiment_7_ok, df_experiment_7_nok, 'Uma única tabela de nome events com paráfrases')
print_percentage_ok_nok(df_experiment_8, df_experiment_8_ok, df_experiment_8_nok, 'Uma única tabela de nome events com paráfrases retirando problemas conhecidos')

Todas as tabelas em única base de dados
OK: 4.88%
NOK: 95.12%
--
Única tabela em uma única base de dados
OK: 31.71%
NOK: 68.29%
--
Cada tabela em uma base de dados diferente
OK: 31.71%
NOK: 68.29%
--
Cada tabela em uma base de dados diferente com ajustes
OK: 57.14%
NOK: 42.86%
--
Cada tabela em uma base de dados diferente com paráfrases
OK: 48.21%
NOK: 51.79%
--
Uma única tabela de nome events com paráfrases
OK: 57.76%
NOK: 42.24%
--
Uma única tabela de nome events com paráfrases retirando problemas conhecidos
OK: 68.7%
NOK: 31.3%
--


## Dataframe com classificação de conceitos das declarações classificadas com fácil

In [7]:
## Experimento 1, 2, 3
df_classification = [
    [2667, 'event', 'events', 'activity', 'count', 0],
    [2670, 'event', 'events', 'resource', 'count', 0],
    [2673, 'resource', 'events', 'none', 'count', 0],
    [2677, 'event', 'events', 'cost', 'projection', 0],
    [2680, 'event', 'events', 'timestamp', 'projection', 0],
    [2681, 'case', 'cases', 'event', 'projection', 1],
    [2689, 'case', 'cases', 'event', 'projection', 0],
    [2697, 'cost', 'events', 'none', 'max', 0],
    [2710, 'event', 'events', 'timestamp', 'count', 0],
    [2716, 'event', 'events', 'cost', 'count', 1],
    [2741, 'event', 'events', 'activity', 'projection', 0],
    [2747, 'activity', 'events', 'cost', 'projection', 0],
    [2751, 'event', 'events', 'case', 'projection', 0],
    [2752, 'event', 'events', 'activity', 'count', 1],
    [2753, 'cost', 'events', 'none', 'max', 0],
    [2757, 'cost', 'events', 'resource', 'sum', 0],
    [2762, 'case', 'cases', 'resource', 'projection', 0],
    [2767, 'event', 'events', 'resource', 'count', 0],
    [2769, 'event', 'events', 'case', 'count', 0],
    [2774, 'case', 'cases', 'activity', 'count', 0],
    [2777, 'case', 'cases', 'cost', 'count', 0],
    [2779, 'case', 'cases', 'resource', 'projection', 0],
    [2780, 'event', 'events', 'resource', 'projection', 0],
    [2782, 'case', 'cases', 'resource', 'projection', 1],
    [2784, 'activity', 'events', 'none', 'projection', 0],
    [2785, 'resource', 'events', 'none', 'min', 0],
    [2823, 'event,case', 'cases', 'case', 'count', 0],
    [2838, 'case', 'cases', 'activity', 'count', 0],
    [2853, 'activity', 'events', 'cost', 'projection', 0],
    [2863, 'timestamp', 'cases', 'none', 'min',1],
    [2864, 'resource', 'events', 'resource', 'projection', 0],
    [2866, 'resource', 'cases', 'resource', 'projection', 1],
    [2874, 'case,resource', 'cases', 'resource', 'count', 1],
    [2884, 'case', 'cases', 'cost', 'count', 1],
    [2886, 'event', 'events', 'activity', 'count', 0],
    [2887, 'cost', 'events', 'none', 'avg', 0],
    [2892, 'activity', 'events', 'event', 'projection', 0],
    [2895, 'event', 'events', 'resource', 'count', 0],
    [2896, 'resource', 'events', 'event', 'projection', 1],
    [3003, 'resource', 'events', 'activity', 'projection', 0],
    [3007, 'event', 'events', 'cost', 'projection', 0]
]
df_experiment_3_classification = pd.DataFrame(df_classification, columns=['_id', 'Sel', 'ON', 'Cond_Group', 'Type', 'Synonym'])

In [17]:
## Experimento 5, 6
df_classification = [
    [2667, 'event', 'events', 'activity', 'count', 0],
    [2670, 'event', 'events', 'resource', 'count', 0],
    [2673, 'resource', 'events', 'none', 'count', 0],
    [2677, 'event', 'events', 'cost', 'projection', 0],
    [2680, 'event', 'events', 'timestamp', 'projection', 0],
    [2681, 'case', 'cases', 'event', 'projection', 1],
    [2689, 'case', 'cases', 'event', 'projection', 0],
    [2697, 'cost', 'events', 'none', 'max', 0],
    [2710, 'event', 'events', 'timestamp', 'count', 0],
    [2716, 'event', 'events', 'cost', 'count', 1],
    [2741, 'event', 'events', 'activity', 'projection', 0],
    [2747, 'activity', 'events', 'cost', 'projection', 0],
    [2751, 'event', 'events', 'case', 'projection', 0],
    [2752, 'event', 'events', 'activity', 'count', 1],
    [2753, 'cost', 'events', 'none', 'max', 0],
    [2757, 'cost', 'events', 'resource', 'sum', 0],
    [2762, 'case', 'cases', 'resource', 'projection', 0],
    [2767, 'event', 'events', 'resource', 'count', 0],
    [2769, 'event', 'events', 'case', 'count', 0],
    [2774, 'case', 'cases', 'activity', 'count', 0],
    [2779, 'case', 'cases', 'resource', 'projection', 0],
    [2780, 'event', 'events', 'resource', 'projection', 0],
    [2782, 'case', 'cases', 'resource', 'projection', 1],
    [2784, 'activity', 'events', 'none', 'projection', 0],
    [2785, 'resource', 'events', 'none', 'min', 0],
    [2838, 'case', 'cases', 'activity', 'count', 0],
    [2853, 'activity', 'events', 'cost', 'projection', 0],
    [2863, 'timestamp', 'cases', 'none', 'min',1],
    [2864, 'resource', 'events', 'resource', 'projection', 0],
    [2886, 'event', 'events', 'activity', 'count', 0],
    [2887, 'cost', 'events', 'none', 'avg', 0],
    [2892, 'activity', 'events', 'event', 'projection', 0],
    [2895, 'event', 'events', 'resource', 'count', 0],
    [3003, 'resource', 'cases', 'activity', 'projection', 0],
    [3007, 'event', 'events', 'cost', 'projection', 0]
]
df_experiment_5_classification = pd.DataFrame(df_classification, columns=['_id', 'Sel', 'ON', 'Cond_Group', 'Type', 'Synonym'])

## Análise do experimento 1

In [9]:
bw_experiment_1, qt_bw_experiment_1, df_bw_experiment_1 = questions_bag_words(df_experiment_1)
df_experiment_1_qt = pd.concat([df_experiment_1.loc[:,["question_id", "question", "exact_match"]].reset_index(drop=True), df_bw_experiment_1.reset_index(drop=True)], axis=1)
df_experiment_1_qt.query('how != 1 and what != 1 and which != 1 and when != 1')
df_experiment_1_qt.query('when == 1')

,question_id,question,exact_match,112,20,2016,50,56300,974,activities,...,were,what,when,where,which,whose,with,working,year,yes
29,2863,When is the oldest problem?,NOK,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [10]:
sum_experiment_1 = qt_bw_experiment_1.sum(axis=0)
words_freq = [(word, sum_experiment_1[0, idx]) for word, idx in bw_experiment_1.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq

[('the', 24),
 ('how', 13),
 ('many', 13),
 ('have', 12),
 ('are', 11),
 ('events', 10),
 ('what', 10),
 ('is', 10),
 ('which', 10),
 ('in', 9),
 ('event', 9),
 ('activities', 8),
 ('resource', 8),
 ('of', 7),
 ('cost', 7),
 ('cases', 7),
 ('log', 6),
 ('null', 6),
 ('not', 5),
 ('as', 4),
 ('resources', 4),
 ('activity', 4),
 ('costevent', 4),
 ('closed', 3),
 ('do', 3),
 ('that', 3),
 ('there', 3),
 ('incidents', 3),
 ('with', 3),
 ('more', 3),
 ('than', 3),
 ('on', 3),
 ('all', 3),
 ('for', 3),
 ('used', 3),
 ('were', 3),
 ('show', 2),
 ('peter', 2),
 ('where', 2),
 ('value', 2),
 ('select', 2),
 ('requests', 2),
 ('any', 2),
 ('records', 2),
 ('idcase', 2),
 ('was', 2),
 ('status', 2),
 ('people', 2),
 ('some', 2),
 ('to', 2),
 ('we', 1),
 ('amount', 1),
 ('had', 1),
 ('different', 1),
 ('took', 1),
 ('place', 1),
 ('year', 1),
 ('2016', 1),
 ('number', 1),
 ('between', 1),
 ('and', 1),
 ('20', 1),
 ('update', 1),
 ('50', 1),
 ('largest', 1),
 ('found', 1),
 ('count', 1),
 ('no', 1

In [11]:
df_experiment_1_cls = pd.concat([df_experiment_1.loc[:,["question_id", "question", "exact_match", "gold"]].reset_index(drop=True), df_experiment_3_classification.reset_index(drop=True)], axis=1)
df_experiment_1_cls

,question_id,question,exact_match,gold,_id,Sel,ON,Cond_Group,Type,Synonym
0,2667,How many closed activities do we have?,NOK,SELECT count(*) FROM log1 WHERE activity = 'Closed',2667,event,events,activity,count,0
1,2670,Show the amount of events that have had Peter as resource,NOK,SELECT COUNT(idevent) FROM log6 WHERE resource = 'Peter',2670,event,events,resource,count,0
2,2673,How many different resources are there in the log?,NOK,SELECT COUNT(DISTINCT resource) FROM log5,2673,resource,events,none,count,0
3,2677,What are the events where the event cost is not null?,NOK,SELECT * FROM log6 WHERE costevent != 'null',2677,event,events,cost,projection,0
4,2680,What are the events that took place in the year 2016?,NOK,SELECT * FROM log1 WHERE enddatetime BETWEEN '2016-01-01' AND '2017-01-31',2680,event,events,timestamp,projection,0
5,2681,What are the incidents with the number of cases between 1 and 20 update events?,NOK,SELECT idcase FROM log3 GROUP BY idcase HAVING count(idcase) BETWEEN 1 AND 20,2681,case,cases,event,projection,1
6,2689,Which cases have more than 50 activities?,NOK,SELECT count(activity) FROM log7 GROUP BY idcase HAVING count(activity) > 50,2689,case,cases,event,projection,0
7,2697,What is the largest event cost value found in the log?,NOK,SELECT max(costevent) FROM log8,2697,cost,events,none,max,0
8,2710,select event count with no end date,NOK,SELECT count(*) FROM log1 WHERE enddatetime = 'null',2710,event,events,timestamp,count,0
9,2716,Counting requests with event cost information,NOK,SELECT count(idevent) FROM log3 WHERE costevent != 'null',2716,event,events,cost,count,1


## Análise do experimento 2 e 3

In [ ]:
bw_experiment_2, qt_bw_experiment_2, df_bw_experiment_2 = questions_bag_words(df_experiment_2)
df_experiment_2_qt = pd.concat([df_experiment_2.loc[:,["question_id", "question", "exact_match"]].reset_index(drop=True), df_bw_experiment_2.reset_index(drop=True)], axis=1)
df_experiment_2_qt.query('how != 1 and what != 1 and which != 1 and when != 1')
df_experiment_2_qt.query('which == 1')

In [ ]:
df_experiment_3_cls = pd.concat([df_experiment_3.loc[:,["question_id", "question", "exact_match", "gold"]].reset_index(drop=True), df_experiment_3_classification.reset_index(drop=True)], axis=1)
df_experiment_3_cls

## Análise do experimento 5

In [18]:
bw_experiment_5, qt_bw_experiment_5, df_bw_experiment_5 = questions_bag_words(df_experiment_5)
sum_experiment_5 = qt_bw_experiment_5.sum(axis=0)
bw_experiment_5.vocabulary_.items()

dict_items([('how', 50), ('many', 62), ('closed', 26), ('activities', 6), ('do', 38), ('we', 107), ('have', 48), ('show', 86), ('the', 92), ('amount', 11), ('of', 68), ('events', 41), ('that', 91), ('had', 45), ('peter', 72), ('as', 16), ('resource', 81), ('different', 37), ('resources', 82), ('are', 15), ('there', 93), ('in', 53), ('log', 61), ('what', 109), ('where', 111), ('event', 40), ('cost', 30), ('is', 56), ('not', 65), ('null', 66), ('happened', 46), ('year', 115), ('2016', 2), ('incidents', 54), ('with', 114), ('number', 67), ('between', 22), ('and', 13), ('20', 1), ('which', 112), ('cases', 25), ('more', 63), ('than', 90), ('50', 3), ('largest', 58), ('value', 104), ('found', 44), ('select', 84), ('count', 32), ('no', 64), ('end', 39), ('date', 35), ('counting', 33), ('requests', 80), ('information', 55), ('any', 14), ('records', 78), ('activity', 7), ('if', 52), ('yes', 116), ('data', 34), ('costevent', 31), ('values', 105), ('on', 70), ('idcase', 51), ('column', 28), ('the

In [27]:
df_experiment_5_qt = pd.concat([df_experiment_5.loc[:,["question_id", "question", "exact_match"]].reset_index(drop=True), df_bw_experiment_5.reset_index(drop=True)], axis=1)
df_experiment_5_qt.query('which == 0 and what == 0 and how == 0 and when == 0')
df_experiment_5_qt

,question_id,question,exact_match,112,20,2016,50,56300,974,activities,...,we,were,what,when,where,which,whose,with,year,yes
0,2667,How many closed activities do we have?,OK,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,2670,Show the amount of events that have had Peter as resource,OK,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2673,How many different resources are there in the log?,OK,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2677,What are the events where the event cost is not null?,OK,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4,2680,What are the events that happened in the year 2016?,NOK,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
5,2681,What are the incidents with the number of events between 1 and 20?,NOK,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
6,2689,Which cases have more than 50 activities?,OK,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
7,2697,What is the largest event cost value found in the log?,NOK,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8,2710,select event count with no end date,NOK,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,2716,Counting requests with event cost information,NOK,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [39]:
df_experiment_5_cls = pd.concat([df_experiment_5.loc[:,["question_id", "question", "exact_match", "predicted", "gold"]].reset_index(drop=True), df_experiment_5_classification.reset_index(drop=True)], axis=1)
df_experiment_5_cls.query('Type == "count"')

,question_id,question,exact_match,predicted,gold,_id,Sel,ON,Cond_Group,Type,Synonym
0,2667,How many closed activities do we have?,OK,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',2667,event,events,activity,count,0
1,2670,Show the amount of events that have had Peter as resource,OK,SELECT Count(*) FROM log6 WHERE log6.resource = 'terminal',SELECT COUNT(*) FROM log6 WHERE resource = 'Peter',2670,event,events,resource,count,0
2,2673,How many different resources are there in the log?,OK,SELECT Count(DISTINCT log5.resource) FROM log5,SELECT COUNT(DISTINCT resource) FROM log5,2673,resource,events,none,count,0
8,2710,select event count with no end date,NOK,SELECT Count(*) FROM log1 WHERE log1.enddatetime NOT IN (SELECT log1.idevent FROM log1),SELECT count(*) FROM log1 WHERE enddatetime = 'null',2710,event,events,timestamp,count,0
9,2716,Counting requests with event cost information,NOK,SELECT Count(*) FROM log3 WHERE log3.costevent = 'terminal',SELECT count(*) FROM log3 WHERE costevent != 'null',2716,event,events,cost,count,1
13,2752,How many records do they have as a send confirmation receipt activity?,OK,SELECT Count(*) FROM log7 WHERE log7.activity = 'terminal',SELECT count(*) FROM log7 WHERE activity = 'send confirmation receipt',2752,event,events,activity,count,1
17,2767,"How many times Siebel, as resource, was used ?",OK,SELECT Count(*) FROM log4 WHERE log4.resource = 'terminal',SELECT count(*) FROM log4 WHERE resource = 'Siebel',2767,event,events,resource,count,0
18,2769,How many events has the case declaration 56300,OK,SELECT Count(*) FROM log8 WHERE log8.idcase = 'terminal',SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',2769,event,events,case,count,0
19,2774,How many cases have the activity status Awaiting Assignment?,NOK,SELECT Count(*) FROM log6 WHERE log6.activity = 'terminal',SELECT count(DISTINCT idcase) FROM log6 WHERE activity = 'Awaiting Assignment',2774,case,cases,activity,count,0
25,2838,How many cases have activity 'in progress'?,NOK,SELECT Count(*) FROM log6 WHERE log6.activity = 'terminal',SELECT COUNT(DISTINCT idcase) FROM log6 WHERE activity = 'In Progress',2838,case,cases,activity,count,0


In [ ]:
np.setdiff1d(df_experiment_1.loc[:,"question_id"],df_experiment_5.loc[:, "question_id"])

In [105]:
vec = TfidfVectorizer(stop_words="english", ngram_range = (1,2))
vec.fit(df_experiment_5["gold"].values)
features = vec.transform(df_experiment_5["gold"].values)

In [120]:
clust = KMeans(init='k-means++', n_clusters=3, n_init=10);
clust.fit(features)

KMeans(n_clusters=3)

In [121]:
yhat=clust.predict(features)

In [123]:
df_experiment_5_k = df_experiment_5.loc[:,["gold", "question"]]
df_experiment_5_k["cluster_labels"] = clust.labels_
df_experiment_5_k.query("cluster_labels == 2")

,gold,question,cluster_labels
0,SELECT count(*) FROM log1 WHERE activity = 'Closed',How many closed activities do we have?,2
3,SELECT COUNT(*) FROM log6 WHERE resource = 'Peter',Show the amount of events that have had Peter as resource,2
6,SELECT COUNT(DISTINCT resource) FROM log5,How many different resources are there in the log?,2
14,SELECT idcase FROM log1 GROUP BY idcase HAVING count(*) BETWEEN 1 AND 20,What are the incidents with the number of events between 1 and 20?,2
18,SELECT idcase FROM log7 GROUP BY idcase HAVING count(*) > 50,Which cases have more than 50 activities?,2
53,SELECT * FROM log6 WHERE activity = 'Closed',"Are there any records of Closed activity in the log? If yes, show event data.",2
62,SELECT count(*) FROM log7 WHERE activity = 'send confirmation receipt',How many records do they have as a send confirmation receipt activity?,2
75,SELECT count(*) FROM log4 WHERE resource = 'Siebel',"How many times Siebel, as resource, was used ?",2
76,SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',How many events has the case declaration 56300,2
81,SELECT count(DISTINCT idcase) FROM log6 WHERE activity = 'Awaiting Assignment',How many cases have the activity status Awaiting Assignment?,2


## Análise do experimento 6

In [131]:
df_experiment_6.iloc[100:150, :]

,question_id,question,predicted,gold,hardness,exact_match,select_match_gold,select_match_pred,select_match,select_match (no_agg)_gold,...,order,and_or_gold,and_or_pred,and_or,IUEN_gold,IUEN_pred,IUEN,keywords_gold,keywords_pred,keywords
100,2767,"How many times Siebel, as resource, was used ?",SELECT Count(*) FROM log4 WHERE log4.resource = 'terminal',SELECT count(*) FROM log4 WHERE resource = 'Siebel',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
101,2767-1,How many times Siebel performed some activity?,SELECT Count(*) FROM log4 WHERE log4.activity = 'terminal',SELECT count(*) FROM log4 WHERE resource = 'Siebel',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
102,2767-2,How many occurrences of Siebel are in the log?,SELECT Count(*) FROM log4 WHERE log4.idcase = 'terminal',SELECT count(*) FROM log4 WHERE resource = 'Siebel',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
103,2769,How many events has the case declaration 56300?,SELECT Count(*) FROM log8 WHERE log8.idcase = 'terminal',SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
104,2769-1,How many events has the case 'declaration 56300'?,SELECT Count(*) FROM log8 WHERE log8.idcase = 'terminal',SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
105,2769-2,How many events has 'declaration 56300'?,SELECT Count(*) FROM log8 WHERE log8.idevent LIKE 'terminal',SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,2.0,NOK
106,2769-3,How many events occurred in the case 'declaration 56300'?,SELECT Count(*) FROM log8 WHERE log8.idcase = 'terminal',SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
107,2769-4,What is the size of the trace associated with the case 'declaration 56300'?,SELECT * FROM log8 WHERE log8.idcase = 'terminal',SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',easy,NOK,1.0,1.0,NOK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
108,2769-5,How many events does the 'declaration 56300' case have?,SELECT Count(*) FROM log8 WHERE log8.idcase = 'terminal',SELECT count(*) FROM log8 WHERE idcase = 'declaration 56300',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
109,2774,How many cases have the activity status Awaiting Assignment?,SELECT Count(*) FROM log6 WHERE log6.activity = 'terminal',SELECT count(DISTINCT idcase) FROM log6 WHERE activity = 'Awaiting Assignment',easy,NOK,1.0,1.0,NOK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK


In [257]:
df_experiment_6.query('question_id.str.contains("-") & exact_match == "NOK"')

,question_id,question,predicted,gold,hardness,exact_match,select_match_gold,select_match_pred,select_match,select_match (no_agg)_gold,...,order,and_or_gold,and_or_pred,and_or,IUEN_gold,IUEN_pred,IUEN,keywords_gold,keywords_pred,keywords
14,2670-3,How many times has employee 'Peter' performed an activity?,SELECT Count(*) FROM log6 WHERE log6.idcase = 'terminal',SELECT COUNT(*) FROM log6 WHERE resource = 'Peter',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
15,2670-4,How many events did employee 'Peter' run?,SELECT Count(*) FROM log6 WHERE log6.activity = 'terminal',SELECT COUNT(*) FROM log6 WHERE resource = 'Peter',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
17,2670-6,Report the number of times employee 'Peter' was responsible for an activity.,SELECT Count(*) FROM log6 WHERE log6.idcase = 'terminal',SELECT COUNT(*) FROM log6 WHERE resource = 'Peter',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
22,2673-3,How many resources are in the event log?,SELECT Count(*) FROM log5,SELECT COUNT(DISTINCT resource) FROM log5,easy,NOK,1.0,1.0,NOK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,0.0,0.0,NaN
23,2673-4,How many different people performed activities?,SELECT Count(*) FROM log5,SELECT COUNT(DISTINCT resource) FROM log5,easy,NOK,1.0,1.0,NOK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,2895-2,How many events Peter was associated?,SELECT Count(*) FROM log6 WHERE log6.idcase = 'terminal',SELECT Count(*) FROM log6 WHERE resource = 'Peter',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
163,3003-1,What resources were associated with closing an incident?,SELECT log6.resource FROM log6,SELECT DISTINCT resource FROM log6 WHERE activity = 'Closed',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,0.0,ND
164,3003-2,What resources worked on closing an incident?,SELECT log6.resource FROM log6,SELECT DISTINCT resource FROM log6 WHERE activity = 'Closed',easy,NOK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,0.0,ND
166,3007-1,Report the events where cost information is non-null value.,SELECT log2.costevent FROM log2 WHERE log2.costevent != 'terminal',SELECT * FROM log2 WHERE costevent != 'null',easy,NOK,1.0,1.0,NOK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK


In [ ]:
bw_experiment_6, qt_bw_experiment_6, df_bw_experiment_6 = questions_bag_words(df_experiment_6)
df_experiment_6_qt = pd.concat([df_experiment_6.loc[:,["question_id", "question", "predicted", "gold", "exact_match"]].reset_index(drop=True), df_bw_experiment_6.reset_index(drop=True)], axis=1)
df_experiment_6_qt.query('what == 1 & which == 1  & question_id.str.contains("-") & exact_match == "NOK"')

In [258]:
df_question_id_f = df_experiment_6.loc[:, 'question_id'].str.split('-', 1, expand=True).iloc[:, 0]
df_experiment_6_qt = pd.concat([df_experiment_6.loc[:,["question_id", "exact_match", "question", "gold", "predicted"]].reset_index(drop=True), df_question_id_f.reset_index(drop=True)], axis=1)
df_experiment_6_qt.rename(columns={0:'_id'}, inplace=True)
df_experiment_6_qt['_id'] = df_experiment_6_qt['_id'].astype('int64')

In [274]:
df_experiment_6_qt = pd.concat([df_experiment_6.loc[:,["question_id", "exact_match"]].reset_index(drop=True), df_question_id_f.reset_index(drop=True)], axis=1)
df_experiment_6_qt_f = df_experiment_6_qt.query('question_id.str.contains("-")')
df_experiment_6_qt_f.groupby([0, "exact_match"]).count()

question_id
0    exact_match             
2667 OK                    10
2670 NOK                    3
     OK                     4
2673 NOK                    4
     OK                     6
2677 NOK                    9
2680 NOK                    4
2681 NOK                    6
2689 OK                     3
2697 NOK                    4
     OK                     1
2710 NOK                    5
     OK                     1
2716 NOK                    3
     OK                     2
2741 NOK                    1
     OK                     2
2747 NOK                    2
     OK                     1
2751 NOK                    1
2752 OK                     3
2753 NOK                    2
     OK                     1
2757 NOK                    1
     OK                     1
2762 NOK                    2
     OK                     1
2767 NOK                    2
2769 NOK                    2
     OK                     3
2774 NOK                    2
2779 OK                     2
2780 NOK                    5
2782 NOK                    1
     OK                     2
2784 OK                     1
2785 NOK                    2
     OK                     1
2838 NOK                    1
2853 NOK                    1
     OK                     1
2863 NOK                    4
2864 OK                     5
2886 OK                     2
2887 OK                     6
2892 OK                     1
2895 NOK                    1
     OK                     1
3003 NOK                    2
3007 NOK                    2

In [264]:
df_experiment_6.query('question_id.str.contains("-")').loc[:, ["question_id", "exact_match"]]

,question_id,exact_match
1,2667-1,OK
2,2667-2,OK
3,2667-3,OK
4,2667-4,OK
5,2667-5,OK
...,...,...
161,2895-2,NOK
163,3003-1,NOK
164,3003-2,NOK
166,3007-1,NOK


In [276]:
df_experiment_6.query('question_id.str.contains("-")').head(50)

,question_id,question,predicted,gold,hardness,exact_match,select_match_gold,select_match_pred,select_match,select_match (no_agg)_gold,...,order,and_or_gold,and_or_pred,and_or,IUEN_gold,IUEN_pred,IUEN,keywords_gold,keywords_pred,keywords
1,2667-1,How many activities 'closed' do we have?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
2,2667-2,How many activities 'closed' are there in the event log?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
3,2667-3,How many occurrences of activity 'closed' are there in the event log?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
4,2667-4,How many events related to activity 'closed' are there in the event log?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
5,2667-5,How many times did activity 'closed' run?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
6,2667-6,How many times did activity 'closed' occur?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
7,2667-7,How many times does the activity named 'closed' run?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
8,2667-8,Report the number of times activity 'closed' was executed.,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
9,2667-9,Report the number of occurrences of activity 'closed'.,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
10,2667-10,Report the number of occurrences of activity named 'closed'.,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK


In [250]:
question_id_diff(df_experiment_6.query('~question_id.str.contains("-") & exact_match == "NOK"'), df_experiment_5_ok)

['2680' '2681' '2697' '2710' '2716' '2753' '2762' '2774' '2780' '2782'
 '2785' '2838' '2863' '2895' '3003']


In [151]:
df_experiment_5_classification.dtypes
df_experiment_6_qt.dtypes

question_id    object
exact_match    object
question       object
gold           object
predicted      object
_id            object
dtype: object

In [252]:
df_experiment_6_qt
df_experiment_6_cls = pd.merge(df_experiment_6_qt.query('~question_id.str.contains("-")'), df_experiment_5_classification, how="left", on="_id")

In [261]:
df_experiment_6_cls.query("exact_match == 'OK'")

,question_id,exact_match,question,gold,predicted,_id,Sel,ON,Cond_Group,Type,Synonym
0,2667,OK,How many closed activities do we have?,SELECT count(*) FROM log1 WHERE activity = 'Closed',SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',2667,event,events,activity,count,0
1,2670,OK,Show the amount of events that have had Peter as resource.,SELECT COUNT(*) FROM log6 WHERE resource = 'Peter',SELECT Count(*) FROM log6 WHERE log6.resource = 'terminal',2670,event,events,resource,count,0
2,2673,OK,How many different resources are there in the log?,SELECT COUNT(DISTINCT resource) FROM log5,SELECT Count(DISTINCT log5.resource) FROM log5,2673,resource,events,none,count,0
3,2677,OK,What are the events where the event cost is not null?,SELECT * FROM log6 WHERE costevent != 'null',SELECT * FROM log6 WHERE log6.costevent != 'terminal',2677,event,events,cost,projection,0
6,2689,OK,Which cases have more than 50 activities?,SELECT idcase FROM log7 GROUP BY idcase HAVING count(*) > 50,SELECT log7.idcase FROM log7 GROUP BY log7.idcase HAVING Count(*) > 'terminal',2689,case,cases,event,projection,0
10,2741,OK,"Are there any records of 'Closed' activity in the log? If yes, show event data.",SELECT * FROM log6 WHERE activity = 'Closed',SELECT * FROM log6 WHERE log6.activity = 'terminal',2741,event,events,activity,projection,0
11,2747,OK,Which activities have the costevent null?,SELECT DISTINCT activity FROM log2 WHERE costevent = 'null',SELECT log2.activity FROM log2 WHERE log2.costevent = 'terminal',2747,activity,events,cost,projection,0
12,2751,OK,Is there Null values on idcase column?,SELECT * FROM log3 WHERE idcase = 'null',SELECT DISTINCT * FROM log3 WHERE log3.idcase = 'terminal',2751,event,events,case,projection,0
13,2752,OK,How many records do they have as a send confirmation receipt activity?,SELECT count(*) FROM log7 WHERE activity = 'send confirmation receipt',SELECT Count(*) FROM log7 WHERE log7.activity = 'terminal',2752,event,events,activity,count,1
15,2757,OK,What is the total of costevent for TVMF77077810579481 resource?,SELECT SUM(costevent) FROM log2 WHERE resource = 'TVMF77077810579481',SELECT Sum(log2.costevent) FROM log2 WHERE log2.resource = 'terminal',2757,cost,events,resource,sum,0


In [174]:
df_experiment_6_cls.shape

(133, 11)

## Análises exploratórias

In [13]:
df_experiment_5.query('hardness == "easy" and exact_match == "OK"').head(50)

,question_id,question,predicted,gold,hardness,exact_match,select_match_gold,select_match_pred,select_match,select_match (no_agg)_gold,...,order,and_or_gold,and_or_pred,and_or,IUEN_gold,IUEN_pred,IUEN,keywords_gold,keywords_pred,keywords
0,2667,How many closed activities do we have?,SELECT Count(*) FROM log1 WHERE log1.activity = 'terminal',SELECT count(*) FROM log1 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
3,2670,Show the amount of events that have had Peter as resource,SELECT Count(*) FROM log6 WHERE log6.resource = 'terminal',SELECT COUNT(*) FROM log6 WHERE resource = 'Peter',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
6,2673,How many different resources are there in the log?,SELECT Count(DISTINCT log5.resource) FROM log5,SELECT COUNT(DISTINCT resource) FROM log5,easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,0.0,0.0,NaN
10,2677,What are the events where the event cost is not null?,SELECT * FROM log6 WHERE log6.costevent != 'terminal',SELECT * FROM log6 WHERE costevent != 'null',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
18,2689,Which cases have more than 50 activities?,SELECT log7.idcase FROM log7 GROUP BY log7.idcase HAVING Count(*) > 'terminal',SELECT idcase FROM log7 GROUP BY idcase HAVING count(*) > 50,easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,2.0,2.0,OK
53,2741,"Are there any records of Closed activity in the log? If yes, show event data.",SELECT * FROM log6 WHERE log6.activity = 'terminal',SELECT * FROM log6 WHERE activity = 'Closed',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
58,2747,Which activities have the costevent null?,SELECT log2.activity FROM log2 WHERE log2.costevent = 'terminal',SELECT DISTINCT activity FROM log2 WHERE costevent = 'null',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
61,2751,Is there Null values on idcase column?,SELECT DISTINCT * FROM log3 WHERE log3.idcase = 'terminal',SELECT * FROM log3 WHERE idcase = 'null',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
62,2752,How many records do they have as a send confirmation receipt activity?,SELECT Count(*) FROM log7 WHERE log7.activity = 'terminal',SELECT count(*) FROM log7 WHERE activity = 'send confirmation receipt',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK
66,2757,What is the total of costevent for TVMF77077810579481 resource?,SELECT Sum(log2.costevent) FROM log2 WHERE log2.resource = 'terminal',SELECT SUM(costevent) FROM log2 WHERE resource = 'TVMF77077810579481',easy,OK,1.0,1.0,OK,1.0,...,NaN,1.0,1.0,OK,0.0,0.0,NaN,1.0,1.0,OK


- 2668 -> activity nao mencionada explicitamente
- 2669 -> OK
- 2671 -> cases, timestamp
- 2674 -> feature(??) -> recurso --> similara a 2669
- 2675 -> OK
- 2688 -> How many activities 'Awaiting Assignment' for each resource
- 2694 -> Ajustar para o case 'INC0001287' -> Retrieve information for case 'INC0001287' between start date and end date (Conceito de case)
- 2695 -> Conceito de cases (modelo provavelmente não consegue responder), não sei se faz muito sentido
- 2704 -> Conceito de cases (O custo total do case) --> Retrieves the total cost for each case
- 2707 -> (???)
- 2709 -> Pergunta ambigua, mas query predita contém erro de sintaxe
- 2712 -> Select idcase, idevent sorted by case number and execution start date
- 2713 -> How many activities 'closed' for each resource?
- 2715 -> OK
- 2718 -> (???)
- 2721 -> List case id and how many resources work on a case that contains more than 3 events
- 2723 -> Conceito de cases
- 2725 -> Ajustar a consulta gold
- 2726 -> Temporal envolvendo conceito de case
- 2731 -> Ajustar has activity 'IN Progress' e sinonimo
- 2735 -> Conceitos de data 
- 2737 -> Dominio (reimbursement), conceito de valido (?), para cada case (??)
- 2743 -> Pergunta ambigua, para cada atividade, porém extremamente complexo em resolver com SQL
- 2745 -> activities utilizando como events
- 2746 -> sinonimos (records -> events)
- 2748 -> activity record(?) --> mas não faz muito sentido
- 2750 -> (???) --> activities or events
- 2758 -> Não respondeu, mas não faz muito sentido esta pergunta
- 2759 -> 


